# Naptha Module Creation Cookbook

This cookbook will guide you through creating and publishing your first Naptha module. We'll create a simple agent module, package it, and publish it to the Naptha Hub.

## What are Naptha Modules?

Naptha Modules are the building blocks of multi-agent applications that enable them to run across multiple nodes. There are several types:

- **Agent Modules:** Chat Agents, Task-solving Agents, ReAct Agents, etc.
- **Tool Modules:** Web Search, External API Integration, etc. 
- **Agent Orchestrator Modules:** Organizations of Coding Agents, Social Simulations, etc.
- **Environment Modules:** Group Chats, Information Boards, Auctions, etc.
- **Knowledge Base Modules:** Wikipedia, GitHub, etc.
- **Memory Modules:** Chat History, Task History, etc.
- **Persona Modules:** Social Personas from Twitter data or synthetic Market Personas

## Prerequisites

First, let's install the required dependencies:

In [ ]:
!curl -sSL https://install.python-poetry.org | python3 -
%pip install naptha-sdk

# alternatively, you can install the SDK from the source
!git clone https://github.com/NapthaAI/naptha-sdk
!cd naptha-sdk
!poetry install


## 1. Setting Up the Module Structure

Let's clone the module template repository:

In [ ]:
!git clone https://github.com/NapthaAI/module_template simple_agent
!cd simple_agent
!cp .env.example .env
!poetry install

Let's clone the module template repository. The template provides the following structure:

```
simple_agent/
├── simple_agent/
│   ├── __init__.py
│   ├── configs/
│   │   ├── deployment.json
│   │   └── llm_configs.json
│   ├── run.py
│   └── schemas.py
├── tests/
│   └── __init__.py
├── pyproject.toml
├── poetry.lock
├── README.md
├── LICENSE
└── .env.example
```

Let's first edit our input schema in `schemas.py`:

In [ ]:
from pydantic import BaseModel
from typing import Union, Dict, Any, List, Optional

class InputSchema(BaseModel):
    """Schema for module inputs"""
    tool_name: str
    tool_input_data: Optional[Union[Dict[str, Any], List[Dict[str, Any]], str]] = None

## 2. Writing the Module Code

Edit the main module file `run.py`:

In [ ]:
from dotenv import load_dotenv
from naptha_sdk.schemas import AgentDeployment, AgentRunInput
from typing import Dict
import os
import logging
from datetime import datetime
from naptha_sdk.user import sign_consumer_id

load_dotenv()

logger = logging.getLogger(__name__)

class SimpleAgent:
    def __init__(self, deployment: AgentDeployment):
        self.deployment = deployment

    def process(self, input_text: str) -> str:
        """Process the input text.
        
        Args:
            input_text: The text to process
            
        Returns:
            Processed text
        """
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        logger.info(f"[{timestamp}] Processed: {input_text}")
        return f"[{timestamp}] Processed: {input_text}"

async def run(module_run: Dict, *args, **kwargs):
    module_run = AgentRunInput(**module_run)
    module_run.inputs = InputSchema(**module_run.inputs)
    simple_agent = SimpleAgent(module_run.agent_deployment)

    method = getattr(simple_agent, module_run.inputs.tool_name, None)
    
    return await method(module_run.inputs)

if __name__ == "__main__":
    # Test code
    import asyncio
    from naptha_sdk.client.naptha import Naptha
    from naptha_sdk.configs import setup_module_deployment

    naptha = Naptha()
    
    deployment = asyncio.run(setup_module_deployment("agent", "simple_agent/configs/deployment.json", node_url = os.getenv("NODE_URL"), load_persona_data=False))
    
    input_params = {
        "tool_name": "process",
        "tool_input_data": "Hello Naptha!",
    }

    module_run = {
        "inputs": input_params,
        "deployment": deployment,
        "consumer_id": naptha.user.id,
        "signature": sign_consumer_id(naptha.user.id, os.getenv("PRIVATE_KEY"))
    }

    result = asyncio.run(run(module_run))

    print("Response: ", result)

## 3. Configure the Module

Edit the deployment configuration in `configs/deployment.json`:

In [ ]:
[
    {
        "name": "agent_deployment_1",
        "module": {
            "name": "simple_agent",
            "description": "A simple example agent",
            "parameters": "{tool_name: str, tool_input_data: str}",
            "module_type": "agent",
            "module_version": "v0.1",
            "module_entrypoint": "run.py",
            "execution_type": "package"
        },
        "node": {"ip": "localhost"}
    }
]

## 4. Test Locally

Let's test our module locally first:

In [ ]:
!cd simple_agent && poetry run python run.py

You should see output similar to:
```
[2025-01-24 10:30:15] Processed: Hello Naptha!
```

This shows our module is working correctly - it's taking the input text, adding a timestamp, and returning the processed result.

## 5. Version and Publish

Add version tag and publish to GitHub:

In [ ]:
!cd simple_agent && git add .
!cd simple_agent && git commit -m "Initial module commit"
!cd simple_agent && git tag v0.1 #recommended for Github or IPFS
!cd simple_agent && git push --tags

## 6. Register on Naptha Hub

You can register the module either from GitHub or IPFS:

In [ ]:
# From GitHub:
!naptha publish -r https://github.com/YOUR_USERNAME/simple_agent

# Or from IPFS:
!naptha publish -r

## 7. Verify Registration

Check that your module is registered:

In [ ]:
!naptha agents

## 8. Run the Module

Finally, let's run our published module:

In [ ]:
!naptha run agent:simple_agent -p "tool_name='process' tool_input_data='Hello from Naptha!'"

## Next Steps

Congratulations! You've created and published your first Naptha module.

You can check out our agent for running simple chat with LLMs [here](https://github.com/NapthaAI/simple_chat_agent)

For more information:
- Check out our [documentation](https://docs.naptha.ai/) and [modules here](https://github.com/orgs/NapthaAI/repositories)
- Join our [Discord Community](https://naptha.ai/naptha-community)
- Follow us on [Twitter](https://twitter.com/NapthaAI)
- Star us on [GitHub](https://github.com/NapthaAI)
- Get help with bug reports and feature requests on [GitHub Issues](https://github.com/NapthaAI/naptha-sdk/issues)